In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")
dbutils.widgets.text("environment", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:

# Describe the bronze table to get column names
column_list = spark.sql(f"""DESCRIBE TABLE {catalog_name}.udp_wcm_bronze_realtime.pos_wcm_saleout""").collect()


# Build the dynamic SELECT query with renamed columns
dynamic_query = f"""CREATE OR REPLACE TEMP VIEW temp_pos_wcm_saleout AS SELECT """

for column in column_list:
    clean_col = column[0].replace('/', '_')  # Replace '/' with '_' in column names
    dynamic_query += f"`{column[0]}` AS {clean_col}, "

# Remove trailing comma and add FROM clause with filtering
dynamic_query = dynamic_query.rstrip(', ') + f"""
FROM {catalog_name}.udp_wcm_bronze_realtime.pos_wcm_saleout
WHERE CAST(try_to_timestamp(ProcDate, 'yyyyMMddHHmmss') AS DATE) = DATE('{proc_date}')
"""

# Execute the query
spark.sql(dynamic_query)
# print(dynamic_query)

In [0]:
pos_wcm_saleout_df = pos_wcm_saleout_df \
    .withColumnRenamed("CalendarDay", "calendar_day") \
    .withColumnRenamed("Header_ref_01", "header_ref_01") \
    .withColumnRenamed("Header_ref_02", "header_ref_02") \
    .withColumnRenamed("Header_ref_03", "header_ref_03") \
    .withColumnRenamed("Header_ref_04", "header_ref_04") \
    .withColumnRenamed("Header_ref_05", "header_ref_05") \
    .withColumnRenamed("IsRetry", "is_retry") \
    .withColumnRenamed("MemberCardNo", "member_card_no") \
    .withColumnRenamed("MemberPointsEarn", "member_points_earn") \
    .withColumnRenamed("MemberPointsRedeem", "member_points_redeem") \
    .withColumnRenamed("OrderInfo", "order_info") \
    .withColumnRenamed("PosNo", "pos_no") \
    .withColumnRenamed("ProcDate", "proc_date") \
    .withColumnRenamed("ReceiptNo", "receipt_no") \
    .withColumnRenamed("StoreCode", "store_code") \
    .withColumnRenamed("TranTime", "tran_time") \
    .withColumnRenamed("TransDiscountCouponEntry", "trans_discount_coupon_entry") \
    .withColumnRenamed("TransLine", "trans_line") \
    .withColumnRenamed("TransPaymentEntry", "trans_payment_entry") \
    .withColumnRenamed("VinidCsn", "vinid_csn") \
    .withColumnRenamed("_rescued_data", "_rescued_data")

pos_wcm_saleout_df.createOrReplaceTempView("pos_wcm_saleout_vw")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
  *
FROM pos_wcm_saleout_vw
""")